<a href="https://colab.research.google.com/github/SNJ292/snj292.github.io/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

years = list(range(2012, 2022))
url_start = "https://www.basketball-reference.com/draft/NBA_{}.html"

# We're using the url to populate our file 'draft' with drafts between 2012-2022
for year in years:
  url = url_start.format(year)
  data = requests.get(url)

  with open("draft/{}.html".format(year), "w+")as f:
    f.write(data.text)

In [ ]:

dfs = []
for year in years:
  #dfs  will hold a list of the dataframes of each draft 
  with open("draft/{}.html".format(year)) as f:
    page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose()
    draft_table = soup.find(id="stats")
    draft = pd.read_html(str(draft_table))[0]
    draft.head()

    dfs.append(draft)
dfs

<class 'pandas.core.frame.DataFrame'>


In [ ]:
#What college produces the most point scoring players historically. 

college_points = {}

for draft in dfs:
  print(draft)
  for player in draft.iterrows():

    if(player[1]["PTS"] != "Totals" and player[1]["PTS"] != "PTS"):
      college = player[1]["College"]
      points = float(player[1]["PTS"])

      if(college != "NaN"):
        if(college_points.__contains__(college) == True):
          college_points[college] = college_points[college] + points

        elif(college_points.__contains__(college) == False):
          college_points[college] = points

#print(college_points)
plt.bar(range(len(college_points)), list(college_points.values()), align='center')
plt.xticks(range(len(college_points)), list(college_points.keys()))
print(len(college_points.keys()))




In [ ]:
import string

#What the average points scored per position in the draft.

names = list(string.ascii_lowercase)
url_pos = "https://www.basketball-reference.com/players/{}.html"

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
